In [ ]:
!pip install langchain langchain_groq langchain_community pypdf cohere

In [ ]:
import os
import cohere
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate


def load_and_process_pdfs(pdf_paths):

    for path in pdf_paths:
        loader = PyPDFLoader(path)
        documents=loader.load()


    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return text_splitter.split_documents(documents)


cohere_api_key = os.environ.get("COHERE_API_KEY")
embeddings = CohereEmbeddings(user_agent="app", model="embed-english-v3.0", cohere_api_key=cohere_api_key)

if not os.path.exists("embeddings"):

  pdf_files = ["quran.pdf", "hadees.pdf"]
  doc_chunks = load_and_process_pdfs(pdf_files)


  vector_store = FAISS.from_documents(doc_chunks, embeddings)
  vector_store.save_local("embeddings")
  retriever = vector_store.as_retriever()
  os.makedirs("embeddings")

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an Islamic scholar (Ustad) providing answers based on the Quran and Hadith.
    Respond with wisdom and reference the Quranic meaning or Hadith when necessary.

    Context: {context}
    Question: {question}

    Answer as an Ustad:
    """
)


llm= ChatGroq(id="DeepSeek-R1-Distill-Llama-70b")
qa_chain = RetrievalQA(llm=llm, retriever=retriever, return_source_documents=True, prompt=prompt_template)


query = "What does Islam say about patience?"
response = qa_chain.run(query)
print(response)
